In [34]:
# import libraries
import pandas as pd

In [35]:
# read data
df = pd.read_csv('best_salesman_homework.csv', parse_dates=[1])

In [36]:
df.head()

,client_account_id,date,event_name,manager_id,manager_nickname
0,0,2022-05-09,first_touch,1.0,Justin Beiber
1,1,2022-03-21,first_touch,3.0,Joe Biden
2,2,2022-04-18,first_touch,2.0,Kylie Jenner
3,3,2022-02-07,first_touch,2.0,Kylie Jenner
4,4,2022-04-08,first_touch,1.0,Justin Beiber


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3183 entries, 0 to 3182
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   client_account_id  3183 non-null   int64         
 1   date               3183 non-null   datetime64[ns]
 2   event_name         3183 non-null   object        
 3   manager_id         3183 non-null   float64       
 4   manager_nickname   3183 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 124.5+ KB


In [38]:
# check how many managers
df['manager_nickname'].value_counts()

Joe Biden        1249
Kylie Jenner     1007
Justin Beiber     927
Name: manager_nickname, dtype: int64

In [39]:
# Total first touchs
first_touch = df[df['event_name'] == 'first_touch']['manager_nickname'].value_counts().sort_index()
first_touch

Joe Biden        1158
Justin Beiber     890
Kylie Jenner      939
Name: manager_nickname, dtype: int64

In [40]:
# Total deals
deal = df[df['event_name'] == 'deal']['manager_nickname'].value_counts().sort_index()
deal

Joe Biden        91
Justin Beiber    37
Kylie Jenner     68
Name: manager_nickname, dtype: int64

In [41]:
# conversion rates
deal/first_touch*100

Joe Biden        7.858377
Justin Beiber    4.157303
Kylie Jenner     7.241747
Name: manager_nickname, dtype: float64

In [42]:
conv_rate = pd.concat([first_touch,deal, deal/first_touch*100], axis=1)
conv_rate.columns = ['first_touchs', 'deals', 'conv_rate']
conv_rate

,first_touchs,deals,conv_rate
Joe Biden,1158,91,7.858377
Justin Beiber,890,37,4.157303
Kylie Jenner,939,68,7.241747


Without any analisys Joe Biden has the best conversion rate = 7.8%

In [43]:
df.loc[39:42, :]

,client_account_id,date,event_name,manager_id,manager_nickname
39,39,2021-06-10,first_touch,3.0,Joe Biden
40,40,2022-05-02,first_touch,1.0,Justin Beiber
41,40,2022-02-24,deal,3.0,Joe Biden
42,41,2022-04-09,first_touch,1.0,Justin Beiber


for client_account_id = 40 we have deal earlier than first touch. It is some mistake. It is nessesary to check how many such mistakes we have. 

In [44]:
# first touch rows
first_touch = df[df['event_name'] == 'first_touch']

In [45]:
# deal rows
deal = df[df['event_name'] == 'deal']

In [46]:
# join deal and first touch rows
deals = pd.concat([deal.set_index('client_account_id'), first_touch.set_index('client_account_id')], axis=1, join='inner')

In [47]:
# rename columns 
deals.columns = ['deal_date', 'deal', 'deal_manager_id', 'deal_manager_nickname', 'ft_date', 'ft_event_name', 'ft_manager_id', 'ft_manager_nickname']

In [48]:
deals

,deal_date,deal,deal_manager_id,deal_manager_nickname,ft_date,ft_event_name,ft_manager_id,ft_manager_nickname
client_account_id,,,,,,,,
40,2022-02-24,deal,3.0,Joe Biden,2022-05-02,first_touch,1.0,Justin Beiber
63,2022-03-14,deal,2.0,Kylie Jenner,2022-02-03,first_touch,1.0,Justin Beiber
86,2022-05-24,deal,1.0,Justin Beiber,2021-12-17,first_touch,3.0,Joe Biden
87,2022-05-23,deal,2.0,Kylie Jenner,2022-05-30,first_touch,1.0,Justin Beiber
127,2021-08-30,deal,3.0,Joe Biden,2022-05-22,first_touch,1.0,Justin Beiber
...,...,...,...,...,...,...,...,...
2793,2022-02-26,deal,2.0,Kylie Jenner,2021-08-08,first_touch,3.0,Joe Biden
2849,2022-01-03,deal,3.0,Joe Biden,2022-04-02,first_touch,3.0,Joe Biden
2856,2021-12-01,deal,2.0,Kylie Jenner,2022-05-30,first_touch,3.0,Joe Biden


In [49]:
# how many deals earlier dates than first touchs
(deals['deal_date'] < deals['ft_date']).sum()

80

There are 80 deals with earlier dates than first touchs. As it is imposible and we don't have explanation how to fix it, let's drop such client account id and calculate conversion rate again 

In [50]:
deals['days_dif'] = (deals['deal_date'] - deals['ft_date']).dt.days

In [51]:
deals[deals['days_dif'] < 0]['deal_manager_nickname'].value_counts()

Joe Biden        50
Kylie Jenner     26
Justin Beiber     4
Name: deal_manager_nickname, dtype: int64

In [52]:
deals[deals['days_dif'] < 0]['ft_manager_nickname'].value_counts()

Justin Beiber    36
Kylie Jenner     28
Joe Biden        16
Name: ft_manager_nickname, dtype: int64

In [53]:
client_id_drop = deals[deals['days_dif'] < 0].index

In [54]:
data = df.query('client_account_id not in @client_id_drop')

In [55]:
# Total first touchs
first_touch = data[data['event_name'] == 'first_touch']['manager_nickname'].value_counts().sort_index()
first_touch

Joe Biden        1142
Justin Beiber     854
Kylie Jenner      911
Name: manager_nickname, dtype: int64

In [56]:
# Total deals
deal = data[data['event_name'] == 'deal']['manager_nickname'].value_counts().sort_index()
deal

Joe Biden        41
Justin Beiber    33
Kylie Jenner     42
Name: manager_nickname, dtype: int64

In [57]:
deal/first_touch*100

Joe Biden        3.590193
Justin Beiber    3.864169
Kylie Jenner     4.610318
Name: manager_nickname, dtype: float64

In [58]:
conv_rate = pd.concat([first_touch,deal, deal/first_touch*100], axis=1)
conv_rate.columns = ['first_touchs', 'deals', 'conv_rate']
conv_rate

,first_touchs,deals,conv_rate
Joe Biden,1142,41,3.590193
Justin Beiber,854,33,3.864169
Kylie Jenner,911,42,4.610318


Now best rate has Kylie Jenner with rate 4.6%